In [1]:
import pandas as pd
import networkx as nx
from networkx.algorithms.community import girvan_newman
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data/shopping_trends.csv')
df.head(100)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,37,Male,Sneakers,Footwear,100,Missouri,L,Pink,Fall,3.8,Yes,PayPal,Free Shipping,Yes,Yes,48,PayPal,Monthly
96,97,32,Male,Gloves,Accessories,73,Delaware,S,Gold,Spring,3.6,Yes,Venmo,Express,Yes,Yes,43,Venmo,Every 3 Months
97,98,21,Male,Shoes,Footwear,92,Virginia,M,Teal,Winter,4.8,Yes,PayPal,Store Pickup,Yes,Yes,37,Cash,Annually
98,99,20,Male,Jeans,Clothing,67,New York,XL,Teal,Spring,2.6,Yes,Cash,Next Day Air,Yes,Yes,20,Bank Transfer,Annually


In [14]:
# Khởi tạo đồ thị
G = nx.Graph()

# Thêm các nút vào đồ thị với thuộc tính từ DataFrame
for index, row in df.iterrows():
    G.add_node(row['Customer ID'], **row.to_dict())

# Bước 1: Nhóm dữ liệu theo Category và Item Purchased, tạo danh sách khách hàng cho mỗi nhóm
product_category_groups = df.groupby(['Category', 'Item Purchased'])['Customer ID'].apply(list)

# Bước 2: Thêm các cạnh vào đồ thị dựa trên cùng một loại sản phẩm trong cùng một danh mục
for (category, item), customers in product_category_groups.items():
    # Nếu có hơn 1 khách hàng mua cùng một loại sản phẩm trong cùng một danh mục, thêm các cạnh giữa tất cả các khách hàng
    if len(customers) > 1:
        for i in range(len(customers)):
            for j in range(i + 1, len(customers)):
                G.add_edge(customers[i], customers[j], relationship='purchased_same_item', category=category, item=item)

# Kiểm tra các cạnh đã thêm
print("Edges in the graph:")
for edge in G.edges(data=True):
    print(edge)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



(3114, 3881, {'relationship': 'purchased_same_item', 'category': 'Clothing', 'item': 'Shirt'})
(3114, 3888, {'relationship': 'purchased_same_item', 'category': 'Clothing', 'item': 'Shirt'})
(3114, 3891, {'relationship': 'purchased_same_item', 'category': 'Clothing', 'item': 'Shirt'})
(3115, 3124, {'relationship': 'purchased_same_item', 'category': 'Accessories', 'item': 'Handbag'})
(3115, 3146, {'relationship': 'purchased_same_item', 'category': 'Accessories', 'item': 'Handbag'})
(3115, 3156, {'relationship': 'purchased_same_item', 'category': 'Accessories', 'item': 'Handbag'})
(3115, 3181, {'relationship': 'purchased_same_item', 'category': 'Accessories', 'item': 'Handbag'})
(3115, 3200, {'relationship': 'purchased_same_item', 'category': 'Accessories', 'item': 'Handbag'})
(3115, 3221, {'relationship': 'purchased_same_item', 'category': 'Accessories', 'item': 'Handbag'})
(3115, 3242, {'relationship': 'purchased_same_item', 'category': 'Accessories', 'item': 'Handbag'})
(3115, 3281, {'

In [4]:
for category, group in df.groupby('Category'):
    customer_ids = group['Customer ID'].tolist()
    for i in range(len(customer_ids)):
        for j in range(i + 1, len(customer_ids)):
            G.add_edge(customer_ids[i], customer_ids[j])

1


In [5]:
# Áp dụng giải thuật Girvan-Newman
communities = girvan_newman(G)

# Lấy kết quả phân hoạch của các cộng đồng
first_level_communities = next(communities)
print(list(first_level_communities))

[{1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17}, {18}, {19}, {20}, {21}, {22}, {23}, {24}, {25}, {26}, {27}, {28}, {29}, {30}, {31}, {32}, {33}, {34}, {35}, {36}, {37}, {38}, {39}, {40}, {41}, {42}, {43}, {44}, {45}, {46}, {47}, {48}, {49}, {50}, {51}, {52}, {53}, {54}, {55}, {56}, {57}, {58}, {59}, {60}, {61}, {62}, {63}, {64}, {65}, {66}, {67}, {68}, {69}, {70}, {71}, {72}, {73}, {74}, {75}, {76}, {77}, {78}, {79}, {80}, {81}, {82}, {83}, {84}, {85}, {86}, {87}, {88}, {89}, {90}, {91}, {92}, {93}, {94}, {95}, {96}, {97}, {98}, {99}, {100}, {101}, {102}, {103}, {104}, {105}, {106}, {107}, {108}, {109}, {110}, {111}, {112}, {113}, {114}, {115}, {116}, {117}, {118}, {119}, {120}, {121}, {122}, {123}, {124}, {125}, {126}, {127}, {128}, {129}, {130}, {131}, {132}, {133}, {134}, {135}, {136}, {137}, {138}, {139}, {140}, {141}, {142}, {143}, {144}, {145}, {146}, {147}, {148}, {149}, {150}, {151}, {152}, {153}, {154}, {155}, {156}, {157}, {158}, {

In [6]:
# for i, community in enumerate(first_level_communities):
#     print(f"Community {i+1}: {list(community)}")


In [7]:
# # Vẽ đồ thị với các cộng đồng được phân biệt bằng màu sắc
# pos = nx.spring_layout(G)
# plt.figure(figsize=(12, 12))
# 
# # Vẽ các cộng đồng
# for community in first_level_communities:
#     nodes = list(community)
#     nx.draw_networkx_nodes(G, pos, nodelist=nodes, node_color='red')
#     nx.draw_networkx_edges(G, pos)
#     nx.draw_networkx_labels(G, pos)
# 
# lt.show()